In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error

In [33]:
# Loading the datasets
turbine_data = pd.read_csv('Preprocessed/Preprocessed_A1_turbine.csv', sep=',', header=0)
synthetic_data = pd.read_csv('Preprocessed/Preprocessed_A1_synthetic.csv', sep=',', header=0)

# Checking the first few rows of each dataset
turbine_data_normalized = turbine_data.head()
synthetic_data_normalized = synthetic_data.head()

In [34]:
turbine_data_normalized.info(), synthetic_data_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   height    5 non-null      float64
 1   fall      5 non-null      float64
 2   net_fall  5 non-null      float64
 3   flow      5 non-null      float64
 4   power     5 non-null      float64
dtypes: float64(5)
memory usage: 332.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v1      5 non-null      float64
 1   v2      5 non-null      float64
 2   v3      5 non-null      float64
 3   v4      5 non-null      float64
 4   v5      5 non-null      float64
 5   v6      5 non-null      float64
 6   v7      5 non-null      float64
 7   v8      5 non-null      float64
 8   v9      5 non-null      float64
 9   z       5 non-null      float64
dtypes: float64(10)
memory usage: 532

(None, None)

In [35]:
turbine_data_normalized.head()

,height,fall,net_fall,flow,power
0,1.094833,1.119178,1.200701,-1.359556,-0.969392
1,1.455974,1.491852,1.562429,-1.359556,-0.918664
2,-0.891441,-0.867176,-0.910882,0.324949,-0.085611
3,-1.162297,-1.146682,-1.182178,0.324949,-0.248735
4,1.636544,1.634400,1.542081,1.167201,2.006047


In [36]:
synthetic_data_normalized.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,z
0,1.280663,-0.793451,1.616521,1.004039,-0.676741,2.553385,-0.466353,-0.674985,1.886479,1.416377
1,0.109896,1.522669,-0.154310,-0.992534,0.008298,1.684637,-0.344116,-0.674985,-0.358487,-0.323709
2,-1.172442,-0.062932,-0.964088,-0.877683,0.109029,1.335112,0.121703,1.481516,-0.967034,-0.820304
3,-1.640673,0.863717,1.460254,0.860615,1.196895,0.572846,0.973119,-0.674985,1.869298,1.715855
4,-1.496693,-1.521392,-0.693229,-1.218331,-0.559655,0.323990,-0.190274,-0.674985,-0.849809,-1.158785


In [37]:
turbine_data_normalized

,height,fall,net_fall,flow,power
0,1.094833,1.119178,1.200701,-1.359556,-0.969392
1,1.455974,1.491852,1.562429,-1.359556,-0.918664
2,-0.891441,-0.867176,-0.910882,0.324949,-0.085611
3,-1.162297,-1.146682,-1.182178,0.324949,-0.248735
4,1.636544,1.634400,1.542081,1.167201,2.006047


In [39]:
# Extract features (X) and target variable (y)
X = turbine_data_normalized.drop('power', axis=1).values
y = turbine_data_normalized['power'].values

In [72]:
class MyNeuralNetwork:            
    def __init__(self, layers, learning_rate, momentum, activation, validation_percentage=0):
        self.L = len(layers)
        self.n = layers.copy()
        self.xi = [np.zeros(l) for l in layers]
        self.w = [np.random.randn(layers[i], layers[i-1]) for i in range(1, self.L)]
        self.theta = [np.zeros(l) for l in layers]
        self.delta = [np.zeros(l) for l in layers]
        self.d_w = [np.zeros_like(w) for w in self.w]
        self.d_theta = [np.zeros_like(t) for t in self.theta]
        self.d_w_prev = [np.zeros_like(w) for w in self.w]
        self.d_theta_prev = [np.zeros_like(t) for t in self.theta]
        self.learning_rate = learning_rate 
        self.momentum = momentum 
        self.activation = activation
        self.validation_percentage = validation_percentage
        self.loss_epochs = []

    def activation_function(self, x):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'linear':
            return x
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Invalid activation function")

    def activation_derivative(self, x):
        if self.activation == 'sigmoid':
            return x * (1 - x)
        elif self.activation == 'relu':
            return np.where(x > 0, 1, 0)
        elif self.activation == 'linear':
            return 1
        elif self.activation == 'tanh':
            return 1 - x**2
        else:
            raise ValueError("Invalid activation function")

    def forward_pass(self, x):
        self.xi[0] = x
        for i in range(1, self.L):
            self.xi[i] = self.activation_function(np.dot(self.w[i-1], self.xi[i-1]) - self.theta[i])

    def backward_pass(self, y):
        self.delta[-1] = (self.xi[-1] - y) * self.activation_derivative(self.xi[-1])
        for i in range(self.L - 2, 0, -1):
            self.delta[i] = np.dot(self.w[i].T, self.delta[i+1]) * self.activation_derivative(self.xi[i])

    def update_weights(self):
        for i in range(1, self.L):
            self.d_w[i-1] = self.learning_rate * np.outer(self.delta[i], self.xi[i-1]) + self.momentum * self.d_w_prev[i-1]
            self.d_theta[i] = self.learning_rate * self.delta[i] + self.momentum * self.d_theta_prev[i]
            self.w[i-1] -= self.d_w[i-1]
            self.theta[i] -= self.d_theta[i]
            self.d_w_prev[i-1] = self.d_w[i-1]
            self.d_theta_prev[i] = self.d_theta[i]
            
    def mape(self, y_true, y_pred):
        mask = y_true != 0
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

    def fit(self, X, y, epochs, batch_size=32, learning_rate_decay=0.1):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.validation_percentage, random_state=42)

        for epoch in range(epochs):
            for _ in range(0, len(X_train), batch_size):
                batch_indices = np.random.choice(len(X_train), min(batch_size, len(X_train)), replace=False)
                if len(batch_indices) < 2:  # Skip batches with less than 2 samples
                    continue

                X_batch, y_batch = X_train[batch_indices], y_train[batch_indices]

                for i in range(len(X_batch)):
                    x_sample, y_sample = X_batch[i], y_batch[i]
                    self.forward_pass(x_sample)
                    self.backward_pass(y_sample)
                    self.update_weights()

            # Calculate metrics with safeguards
            train_predictions = self.predict(X_train)
            val_predictions = self.predict(X_val)

            training_error = mean_squared_error(y_train, train_predictions)
            validation_error = mean_squared_error(y_val, val_predictions)

            training_r2 = r2_score(y_train, train_predictions) if len(y_train) > 1 else None
            validation_r2 = r2_score(y_val, val_predictions) if len(y_val) > 1 else None

            print(f"Epoch {epoch}/{epochs} - Training Error: {training_error}, Validation Error: {validation_error}, Training R2: {training_r2}, Validation R2: {validation_r2}")

            # Learning rate decay
            self.learning_rate *= (1.0 / (1.0 + learning_rate_decay * epoch))

        final_train_mape = self.mape(y_train, self.predict(X_train))
        final_val_mape = self.mape(y_val, self.predict(X_val))
        final_train_r2 = r2_score(y_train, self.predict(X_train))
        final_val_r2 = r2_score(y_val, self.predict(X_val))
        print(f"Final Training MAPE: {final_train_mape}, Final Validation MAPE: {final_val_mape}")
        print(f"Final Training R2: {final_train_r2}")
        print(f"Final Training MSE: {training_error}, Final Validation MSE: {validation_error}")
        return np.array(self.loss_epochs)
    
    
    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            self.forward_pass(X[i])
            predictions.append(self.xi[-1][0])
        return np.array(predictions)

    def get_loss_epochs(self):
        return np.array(self.loss_epochs)

In [73]:
# Define neural network parameters with two hidden layers
layers = [4, 9, 14, 1]  # Input layer: 13 features, Hidden layers: 10, 8, Output layer: 1 unit
learning_rate = 0.001
momentum = 0.9
activation = 'relu'
validation_percentage = 0.2
epochs = 100

In [74]:
# Create and train the neural network
nn = MyNeuralNetwork(layers, learning_rate, momentum, activation, validation_percentage)
loss_history = nn.fit(X, y, epochs)

Epoch 0/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 1/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 2/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 3/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 4/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 5/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None
Epoch 6/100 - Training Error: 1.2582862039476381, Validation Error: 0.8439444287553015, Training R2: -0.02511479268423078, Validation R2: None

C:\Users\pranj\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
